In [1]:
from tensorflow.keras.models import Model
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from PIL import Image
import cv2
import os
import numpy as np
import pandas as pd

dataset_dir = "../StanfordCarDataset"


Preprocessing/Resizing the images 

In [2]:
def normalize_image(image):
  image_array = np.array(image)
  normalized_array = image_array / 255.0
  return Image.fromarray((normalized_array * 255).astype(np.uint8))

def denoise_image(image):
  # Apply median filtering (adjust kernel size as needed)
  image = cv2.medianBlur(image, ksize=5)

  # Normalize the image
  return normalize_image(image)

def preprocess_image(image):
  return normalize_image(image)
  # denoise_image(image)

Preprocessing images by resizing and normalizing them.

Saving the new images in new train and test directories.

In [4]:
train_dataset_folder = f"{dataset_dir}/cars_train/cars_train"
train_output_folder = f"{dataset_dir}/resized_dataset_train"

test_dataset_folder = f"{dataset_dir}/cars_test/cars_test"
test_output_folder = f"{dataset_dir}/resized_dataset_test"

In [3]:
def resize_images(dataset_folder, output_folder):
    # Loop through all images in the dataset folder
    for filename in os.listdir(dataset_folder):
        # Skiping non-image files in case any
        if not filename.lower().endswith((".jpg", ".jpeg", ".png")):
            continue

        image_path = os.path.join(dataset_folder, filename)
        image = Image.open(image_path)

        resized_image = image.resize(target_size, Image.LANCZOS)

        if image.mode != 'RGB':
            resized_image = resized_image.convert('RGB')
            
        resized_image = preprocess_image(resized_image)

        # Saving the resized image
        output_path = os.path.join(output_folder, filename)
        resized_image.save(output_path)

target_size = (224, 224)
output_folder = f"../"

os.makedirs(train_output_folder, exist_ok=True)
os.makedirs(test_output_folder, exist_ok=True)

resize_images(train_dataset_folder, train_output_folder)
resize_images(test_dataset_folder, test_output_folder)

In [9]:
base_model = VGG16(weights='imagenet', include_top=True)
feature_extractor = Model(inputs=base_model.input, outputs=base_model.get_layer('block5_pool').output)

# List to store extracted features
features_list = []

# Iterate over the images in the directory
for filename in os.listdir(train_dataset_folder):
    if filename.endswith(('.jpg', '.jpeg', '.png')):
        # Load and preprocess the image
        img_path = os.path.join(train_dataset_folder, filename)
        img = Image.open(img_path)
        img_array = np.array(img)
        img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
        img_array = img_array / 255.0

        # Extract features from the image
        features = feature_extractor.predict(img_array)

        # Append the features to the list
        features_list.append(features)

# features_array = np.vstack(features_list)
features_list_flattened = [features.reshape(features.shape[0], -1) for features in features_list]

# Convert the numpy array to a pandas DataFrame
features_df = pd.DataFrame(np.concatenate(features_list_flattened, axis=0))

# Save the DataFrame to a CSV file
features_df.to_csv('extracted_features.csv', index=False)

1/1 [==============================] - 1s 1s/step
